In [180]:
%matplotlib tk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, TextBox
from plot_interactions import on_hover, on_click


In [181]:
basename = "2026_02_06/slice_ap"
waveform_basename = "2026_02_06/slice"


In [182]:
# Check which Python interpreter and kernel you're using
import sys
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print(f"Numpy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")


Python executable: /home/lcb/Documents/shim_client/.venv/bin/python
Python version: 3.14.2 (main, Dec  5 2025, 00:00:00) [GCC 15.2.1 20251111 (Red Hat 15.2.1-4)]
Numpy version: 2.4.1
Pandas version: 3.0.0


In [183]:
clock_hz = 20_000_000
sample_period = 1.0 / clock_hz

dfs = [pd.DataFrame() for _ in range(4)]

# Load the trigger file to determine number of triggers
trig_df = pd.read_csv(f"data/{basename}_trig.csv", sep=r"\s+", header=None)
num_slices = len(trig_df)

trig_raw = trig_df.iloc[:, 0].astype(str).tolist() if num_slices > 0 else []
trig_start_cycles = []
for raw in trig_raw:
    raw = raw.strip()
    if raw.startswith("0x") or raw.startswith("0X"):
        trig_start_cycles.append(int(raw, 16))
    else:
        trig_start_cycles.append(int(raw))
trig_start_times = [cycles / clock_hz for cycles in trig_start_cycles]

# Load the files
for i in range(4):
  dfs[i] = pd.read_csv(f"data/{basename}_bd_{i}.csv", sep=r"\s+", header=None)

# Initialize the array
adc_array = dfs[0].to_numpy()

# Append each subsequent DataFrame to the array (in the second dimension)
for i in range(1, 4):
  adc_array = np.concatenate((adc_array, dfs[i].to_numpy()), axis=1)

# Convert ADC values to amps (-2^15 to 2^15 -> -5.0 to 5.0 A)
adc_array = adc_array * (5.0 / 32768)

total_samples = adc_array.shape[0]
num_channels = adc_array.shape[1]


## Helper functions
Small utilities for parsing rdout patterns and slicing ADC data.

In [184]:
def load_rdout_time_pattern(rdout_path, clock_rate_hz):
    trigger_times = []
    current_trigger = -1
    current_time = 0.0

    def ensure_trigger(idx):
        while len(trigger_times) <= idx:
            trigger_times.append([])

    with open(rdout_path, "r") as handle:
        for line in handle:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            parts = line.split()
            cmd = parts[0]
            if cmd == "NT":
                if len(parts) < 2:
                    continue
                step = int(parts[1])
                for _ in range(step):
                    current_trigger += 1
                    ensure_trigger(current_trigger)
                    current_time = 0.0
                continue
            if current_trigger < 0:
                continue
            if cmd in ("D", "ND"):
                if len(parts) < 2:
                    continue
                delay_cycles = int(parts[1])
                repeat = int(parts[2]) if len(parts) > 2 else 0
                count = repeat + 1
                delta = delay_cycles / clock_rate_hz
                if cmd == "D":
                    for _ in range(count):
                        trigger_times[current_trigger].append(current_time)
                        current_time += delta
                else:
                    current_time += delta * count
    total_samples_in_pattern = sum(len(times) for times in trigger_times)
    return trigger_times, total_samples_in_pattern

def load_wfm_time_data(wfm_path, clock_rate_hz, channels_per_board=8):
    trigger_times = []
    trigger_data = []
    current_trigger = -1
    current_time = 0.0

    def ensure_trigger(idx):
        while len(trigger_times) <= idx:
            trigger_times.append([])
            trigger_data.append([])

    with open(wfm_path, "r") as handle:
        for line in handle:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            parts = line.split()
            cmd = parts[0]
            if cmd == "NT":
                if len(parts) < 2:
                    continue
                step = int(parts[1])
                for _ in range(step):
                    current_trigger += 1
                    ensure_trigger(current_trigger)
                    current_time = 0.0
                continue
            if cmd == "T":
                if len(parts) < 2 + channels_per_board:
                    continue
                step = int(parts[1])
                current_trigger += step
                ensure_trigger(current_trigger)
                current_time = 0.0
                samples = [float(x) for x in parts[2:2 + channels_per_board]]
                trigger_times[current_trigger].append(current_time)
                trigger_data[current_trigger].append(samples)
                continue
            if current_trigger < 0:
                continue
            if cmd == "D":
                if len(parts) < 2 + channels_per_board:
                    continue
                delay_cycles = int(parts[1])
                current_time += delay_cycles / clock_rate_hz
                samples = [float(x) for x in parts[2:2 + channels_per_board]]
                trigger_times[current_trigger].append(current_time)
                trigger_data[current_trigger].append(samples)
                continue
            if cmd == "ND":
                if len(parts) < 2:
                    continue
                delay_cycles = int(parts[1])
                current_time += delay_cycles / clock_rate_hz
    return trigger_times, trigger_data

def build_equal_length_slices(data, trigger_count):
    if trigger_count == 0:
        return [data], [data.shape[0]]
    base_len = data.shape[0] // trigger_count
    remainder = data.shape[0] - (base_len * trigger_count)
    if remainder != 0:
        print(f"Warning: {remainder} samples dropped to keep equal trigger lengths")
    lengths = [base_len for _ in range(trigger_count)]
    slices = []
    for i in range(trigger_count):
        start = i * base_len
        end = start + base_len
        slices.append(data[start:end, :])
    return slices, lengths

def build_trigger_times_from_pattern(pattern_times, trigger_count):
    pattern_trigger_count = len(pattern_times)
    if pattern_trigger_count == 0:
        raise ValueError("No triggers parsed from rdout pattern")
    if trigger_count % pattern_trigger_count != 0:
        raise ValueError(
            f"Trigger count {trigger_count} is not a multiple of rdout triggers {pattern_trigger_count}"
        )
    repeat_factor = trigger_count // pattern_trigger_count
    trigger_times = []
    for _ in range(repeat_factor):
        for times in pattern_times:
            trigger_times.append(list(times))
    return trigger_times

def partition_adc_by_lengths(data, lengths):
    slices = []
    cursor = 0
    for length in lengths:
        end = cursor + length
        if end > data.shape[0]:
            raise ValueError("Readout lengths exceed available ADC samples")
        slices.append(data[cursor:end, :])
        cursor = end
    if cursor < data.shape[0]:
        print(f"Warning: {data.shape[0] - cursor} samples dropped after rdout partitioning")
    return slices


## Build triggers
Create time axes and partition ADC samples into trigger slices.

In [185]:
if waveform_basename:
    rdout_path = f"waveforms/{waveform_basename}.rdout"
    pattern_times, pattern_total_samples = load_rdout_time_pattern(rdout_path, clock_hz)
    if pattern_total_samples == 0:
        print("Warning: rdout pattern has zero total samples")
    trigger_times = build_trigger_times_from_pattern(pattern_times, num_slices)
    trigger_lengths = [len(times) for times in trigger_times]
    trigger_slices = partition_adc_by_lengths(adc_array, trigger_lengths)
else:
    trigger_slices, trigger_lengths = build_equal_length_slices(adc_array, num_slices)
    trigger_times = [np.arange(0, length) * sample_period for length in trigger_lengths]

wfm_board_times = []
wfm_board_data = []
if waveform_basename:
    for board_idx in range(4):
        wfm_path = f"waveforms/{waveform_basename}_bd{board_idx}.wfm"
        try:
            board_times, board_data = load_wfm_time_data(wfm_path, clock_hz)
        except FileNotFoundError:
            print(f"Warning: missing waveform file {wfm_path}")
            board_times, board_data = [], []
        if len(board_times) > 0:
            while len(board_times) < num_slices:
                board_times.extend(board_times)
                board_data.extend(board_data)
            board_times = board_times[:num_slices]
            board_data = board_data[:num_slices]
        else:
            board_times = [[] for _ in range(num_slices)]
            board_data = [[] for _ in range(num_slices)]
        board_times = [np.array(times, dtype=float) for times in board_times]
        board_data = [
            np.array(samples, dtype=float) * (5.0 / 32768) if len(samples) else np.empty((0, 8))
            for samples in board_data
        ]
        wfm_board_times.append(board_times)
        wfm_board_data.append(board_data)
else:
    wfm_board_times = None
    wfm_board_data = None

wfm_step_times = None
wfm_step_data = None
if wfm_board_times is not None and wfm_board_data is not None:
    wfm_step_times = []
    wfm_step_data = []
    for board_idx in range(4):
        last_values = np.zeros(8)
        board_step_times = []
        board_step_data = []
        for trigger_idx in range(num_slices):
            times = wfm_board_times[board_idx][trigger_idx]
            data = wfm_board_data[board_idx][trigger_idx]
            if data.size:
                final_values = data[-1]
                times_ext = np.concatenate(([-10.0], times, [10.0]))
                data_ext = np.vstack((last_values, data, final_values))
            else:
                final_values = last_values.copy()
                times_ext = np.array([-10.0, 10.0])
                data_ext = np.vstack((last_values, final_values))
            board_step_times.append(times_ext)
            board_step_data.append(data_ext)
            last_values = final_values
        wfm_step_times.append(board_step_times)
        wfm_step_data.append(board_step_data)

# Calculate global min/max for consistent y-axis limits
global_min = np.min(adc_array)
global_max = np.max(adc_array)
y_range = global_max - global_min
padding = y_range * 0.05
global_ylim = (global_min - padding, global_max + padding)

print(f"Detected {len(trigger_slices)} triggers from trigger file")
print(f"Total samples: {total_samples}")
print(f"Trigger length range: {min(trigger_lengths)} to {max(trigger_lengths)} samples")
print(f"Number of channels: {num_channels}")
print(f"Global y-axis limits: {global_ylim[0]:.3f} to {global_ylim[1]:.3f} A")


Detected 480 triggers from trigger file
Total samples: 89760
Trigger length range: 187 to 187 samples
Number of channels: 32
Global y-axis limits: -2.183 to 2.166 A


## Plot and interaction
Interactive plot with toggle button, sliders, and hover/click behaviors.

In [186]:
# Interactive toggle plot: starts with triggers for channel 0, click to switch modes
fig, ax = plt.subplots(figsize=(14, 7))
fig.subplots_adjust(left=0.1, bottom=0.2)

# Widget axes
# [left, bottom, width, height] in figure-relative coordinates
button_ax = fig.add_axes([0.15, 0.055, 0.18, 0.08])

channel_slider_ax = fig.add_axes([0.35, 0.08, 0.3, 0.03])
channel_minus_ax = fig.add_axes([0.66, 0.075, 0.035, 0.045])
channel_plus_ax = fig.add_axes([0.70, 0.075, 0.035, 0.045])
channel_textbox_ax = fig.add_axes([0.75, 0.07, 0.08, 0.055])

trig_slider_ax = fig.add_axes([0.35, 0.025, 0.3, 0.03])
trig_minus_ax = fig.add_axes([0.66, 0.02, 0.035, 0.045])
trig_plus_ax = fig.add_axes([0.70, 0.02, 0.035, 0.045])
trig_textbox_ax = fig.add_axes([0.75, 0.015, 0.08, 0.055])

chan_slider_ax = fig.add_axes([0.35, 0.025, 0.3, 0.03])
chan_minus_ax = fig.add_axes([0.66, 0.02, 0.035, 0.045])
chan_plus_ax = fig.add_axes([0.70, 0.02, 0.035, 0.045])
chan_textbox_ax = fig.add_axes([0.75, 0.015, 0.08, 0.055])

channel_label = fig.text(0.5, 0.105, "Channel", ha='center', va='center', fontsize=10)
slider_label = fig.text(0.5, 0.005, "Trigger", ha='center', va='center', fontsize=10)

toggle_button = Button(button_ax, "All Triggers: OFF")

channel_slider = Slider(channel_slider_ax, "", 0, num_channels - 1, valinit=0, valstep=1)
channel_slider.label.set_visible(False)
channel_slider.valtext.set_visible(False)
channel_minus_button = Button(channel_minus_ax, "-")
channel_plus_button = Button(channel_plus_ax, "+")
channel_box = TextBox(channel_textbox_ax, "", initial="0")
channel_box.label.set_visible(False)

trig_slider = Slider(trig_slider_ax, "", 0, len(trigger_slices) - 1, valinit=0, valstep=1)
trig_slider.label.set_visible(False)
trig_slider.valtext.set_visible(False)
trig_minus_button = Button(trig_minus_ax, "-")
trig_plus_button = Button(trig_plus_ax, "+")
trig_box = TextBox(trig_textbox_ax, "", initial="0")
trig_box.label.set_visible(False)

chan_slider = Slider(chan_slider_ax, "", 0, len(trigger_slices) - 1, valinit=0, valstep=1)
chan_slider.label.set_visible(False)
chan_slider.valtext.set_visible(False)
chan_minus_button = Button(chan_minus_ax, "-")
chan_plus_button = Button(chan_plus_ax, "+")
chan_box = TextBox(chan_textbox_ax, "", initial="0")
chan_box.label.set_visible(False)

# State tracking
state = {
    'mode': 'triggers',  # 'triggers' or 'channels'
    'channel_idx': 0,    # Current channel when in triggers mode
    'slice_idx': 0,      # Current trigger when in channels mode
    'trigger_idx': 0,    # Current trigger when in single-trigger mode
    'show_all_triggers': False,
    'lines': [],
    'line_indices': [],
    'annotation': None,
    'highlighted_line': [None],
    'toggle_button': toggle_button,
    'channel_slider': channel_slider,
    'channel_minus_button': channel_minus_button,
    'channel_plus_button': channel_plus_button,
    'channel_box': channel_box,
    'trig_slider': trig_slider,
    'trig_minus_button': trig_minus_button,
    'trig_plus_button': trig_plus_button,
    'trig_box': trig_box,
    'chan_slider': chan_slider,
    'chan_minus_button': chan_minus_button,
    'chan_plus_button': chan_plus_button,
    'chan_box': chan_box,
    'button_ax': button_ax,
    'channel_slider_ax': channel_slider_ax,
    'channel_minus_ax': channel_minus_ax,
    'channel_plus_ax': channel_plus_ax,
    'channel_textbox_ax': channel_textbox_ax,
    'trig_slider_ax': trig_slider_ax,
    'trig_minus_ax': trig_minus_ax,
    'trig_plus_ax': trig_plus_ax,
    'trig_textbox_ax': trig_textbox_ax,
    'chan_slider_ax': chan_slider_ax,
    'chan_minus_ax': chan_minus_ax,
    'chan_plus_ax': chan_plus_ax,
    'chan_textbox_ax': chan_textbox_ax,
    'slider_label': slider_label,
    'channel_label': channel_label,
}

def update_toggle_button_style():
    if state['show_all_triggers']:
        state['toggle_button'].label.set_text("All Triggers: ON")
        state['toggle_button'].ax.set_facecolor("#c7e9c0")
    else:
        state['toggle_button'].label.set_text("All Triggers: OFF")
        state['toggle_button'].ax.set_facecolor("#f0f0f0")
    state['toggle_button'].ax.figure.canvas.draw_idle()

def update_widget_visibility():
    show_triggers_mode = state['mode'] == 'triggers'
    show_channels_mode = state['mode'] == 'channels'
    state['button_ax'].set_visible(show_triggers_mode)
    if show_triggers_mode:
        state['channel_slider_ax'].set_visible(True)
        state['channel_minus_ax'].set_visible(True)
        state['channel_plus_ax'].set_visible(True)
        state['channel_textbox_ax'].set_visible(True)
        state['channel_slider'].set_active(True)
        state['channel_minus_button'].set_active(True)
        state['channel_plus_button'].set_active(True)
        state['channel_box'].set_active(True)
    else:
        state['channel_slider_ax'].set_visible(False)
        state['channel_minus_ax'].set_visible(False)
        state['channel_plus_ax'].set_visible(False)
        state['channel_textbox_ax'].set_visible(False)
        state['channel_slider'].set_active(False)
        state['channel_minus_button'].set_active(False)
        state['channel_plus_button'].set_active(False)
        state['channel_box'].set_active(False)
    if show_triggers_mode and not state['show_all_triggers']:
        state['trig_slider_ax'].set_visible(True)
        state['trig_minus_ax'].set_visible(True)
        state['trig_plus_ax'].set_visible(True)
        state['trig_textbox_ax'].set_visible(True)
        state['trig_slider'].set_active(True)
        state['trig_minus_button'].set_active(True)
        state['trig_plus_button'].set_active(True)
        state['trig_box'].set_active(True)
    else:
        state['trig_slider_ax'].set_visible(False)
        state['trig_minus_ax'].set_visible(False)
        state['trig_plus_ax'].set_visible(False)
        state['trig_textbox_ax'].set_visible(False)
        state['trig_slider'].set_active(False)
        state['trig_minus_button'].set_active(False)
        state['trig_plus_button'].set_active(False)
        state['trig_box'].set_active(False)
    if show_channels_mode:
        state['chan_slider_ax'].set_visible(True)
        state['chan_minus_ax'].set_visible(True)
        state['chan_plus_ax'].set_visible(True)
        state['chan_textbox_ax'].set_visible(True)
        state['chan_slider'].set_active(True)
        state['chan_minus_button'].set_active(True)
        state['chan_plus_button'].set_active(True)
        state['chan_box'].set_active(True)
    else:
        state['chan_slider_ax'].set_visible(False)
        state['chan_minus_ax'].set_visible(False)
        state['chan_plus_ax'].set_visible(False)
        state['chan_textbox_ax'].set_visible(False)
        state['chan_slider'].set_active(False)
        state['chan_minus_button'].set_active(False)
        state['chan_plus_button'].set_active(False)
        state['chan_box'].set_active(False)
    state['slider_label'].set_visible((show_triggers_mode and not state['show_all_triggers']) or show_channels_mode)
    state['channel_label'].set_visible(show_triggers_mode)

def sync_trig_textbox_to_trigger():
    if state['trig_box'] is None:
        return
    prev_eventson = state['trig_box'].eventson
    state['trig_box'].eventson = False
    state['trig_box'].set_val(str(state['trigger_idx']))
    state['trig_box'].eventson = prev_eventson

def sync_trig_slider_to_trigger():
    if state['trig_slider'] is None:
        return
    prev_eventson = state['trig_slider'].eventson
    state['trig_slider'].eventson = False
    state['trig_slider'].set_val(state['trigger_idx'])
    state['trig_slider'].eventson = prev_eventson
    sync_trig_textbox_to_trigger()

def sync_channel_textbox_to_idx():
    if state['channel_box'] is None:
        return
    prev_eventson = state['channel_box'].eventson
    state['channel_box'].eventson = False
    state['channel_box'].set_val(str(state['channel_idx']))
    state['channel_box'].eventson = prev_eventson

def sync_channel_slider_to_idx():
    if state['channel_slider'] is None:
        return
    prev_eventson = state['channel_slider'].eventson
    state['channel_slider'].eventson = False
    state['channel_slider'].set_val(state['channel_idx'])
    state['channel_slider'].eventson = prev_eventson
    sync_channel_textbox_to_idx()

def sync_chan_textbox_to_slice():
    if state['chan_box'] is None:
        return
    prev_eventson = state['chan_box'].eventson
    state['chan_box'].eventson = False
    state['chan_box'].set_val(str(state['slice_idx']))
    state['chan_box'].eventson = prev_eventson

def sync_chan_slider_to_slice():
    if state['chan_slider'] is None:
        return
    prev_eventson = state['chan_slider'].eventson
    state['chan_slider'].eventson = False
    state['chan_slider'].set_val(state['slice_idx'])
    state['chan_slider'].eventson = prev_eventson
    sync_chan_textbox_to_slice()

def format_start_time(seconds):
    total_ns = int(round(seconds * 1_000_000_000))
    minutes = total_ns // 60_000_000_000
    total_ns -= minutes * 60_000_000_000
    secs = total_ns // 1_000_000_000
    total_ns -= secs * 1_000_000_000
    millis = total_ns // 1_000_000
    total_ns -= millis * 1_000_000
    micros = total_ns // 1_000
    nanos = total_ns - (micros * 1_000)
    return f"Start: {minutes}m {secs}s {millis}ms {micros}us {nanos}ns"


def get_start_time_text(trigger_idx):
    if trigger_idx < 0 or trigger_idx >= len(trig_start_times):
        return "Start: n/a"
    return format_start_time(trig_start_times[trigger_idx])


def set_plot_titles(main_title, subtitle, start_time_text):
    ax.set_title(main_title, fontsize=14, fontweight='bold', pad=30)
    ax.text(0.5, 1.045, subtitle, transform=ax.transAxes, ha='center', va='bottom', fontsize=10)
    ax.text(0.5, 1.01, start_time_text, transform=ax.transAxes, ha='center', va='bottom', fontsize=10)

def get_adc_time_limits(time_slice):
    if time_slice is None or len(time_slice) == 0:
        return 0.0, 0.001
    return float(time_slice[0]), float(time_slice[-1])

def get_adc_time_limits_all(time_slices):
    min_time = None
    max_time = None
    for times in time_slices:
        if len(times) == 0:
            continue
        start = float(times[0])
        end = float(times[-1])
        min_time = start if min_time is None else min(min_time, start)
        max_time = end if max_time is None else max(max_time, end)
    if min_time is None or max_time is None:
        return 0.0, 0.001
    return min_time, max_time


def update_plot():
    """Redraw the plot based on current state"""
    ax.clear()
    state['lines'] = []
    state['line_indices'] = []
    
    if state['mode'] == 'triggers':
        # Show triggers for current channel
        if state['show_all_triggers']:
            for trigger_idx in range(len(trigger_slices)):
                slice_data = trigger_slices[trigger_idx]
                time_slice = trigger_times[trigger_idx]
                line, = ax.plot(time_slice, slice_data[:, state['channel_idx']], alpha=0.5, linewidth=2)
                state['lines'].append(line)
                state['line_indices'].append(trigger_idx)
            
            set_plot_titles(
                f"Channel {state['channel_idx']} - All Triggers",
                "Hover to identify, click to view that trigger's channels",
                " "
            )
            x_start, x_end = get_adc_time_limits_all(trigger_times)
        else:
            trigger_idx = state['trigger_idx']
            slice_data = trigger_slices[trigger_idx]
            time_slice = trigger_times[trigger_idx]
            time_slice_ms = np.array(time_slice, dtype=float) * 1000.0
            line, = ax.plot(
                time_slice_ms,
                slice_data[:, state['channel_idx']],
                alpha=0.8,
                linewidth=2,
                label="ADC data"
            )
            state['lines'].append(line)
            state['line_indices'].append(trigger_idx)

            target_plotted = False
            if wfm_step_times is not None and wfm_step_data is not None:
                board_idx = state['channel_idx'] // 8
                channel_in_board = state['channel_idx'] % 8
                if board_idx < len(wfm_step_times) and trigger_idx < len(wfm_step_times[board_idx]):
                    target_times = wfm_step_times[board_idx][trigger_idx]
                    target_samples = wfm_step_data[board_idx][trigger_idx]
                    if target_samples.size and target_times.size:
                        ax.step(
                            target_times * 1000.0,
                            target_samples[:, channel_in_board],
                            where="post",
                            linestyle=":",
                            linewidth=2,
                            color="tab:orange",
                            label="Target DAC"
                        )
                        target_plotted = True
            
            set_plot_titles(
                f"Channel {state['channel_idx']} - Trigger {trigger_idx}",
                "Hover to identify, click to view that trigger's channels",
                get_start_time_text(trigger_idx)
            )
            if target_plotted:
                ax.legend(loc="best")
            x_start, x_end = get_adc_time_limits(time_slice)
            x_start *= 1000.0
            x_end *= 1000.0
        
        ax.set_xlabel('Time (ms)')
        ax.set_ylabel('Current (A)')
        ax.grid(True)
        x_range = x_end - x_start
        ax.set_xlim(x_start - 0.05 * x_range, x_end + 0.05 * x_range)
        
    else:  # mode == 'channels'
        # Show all channels for current trigger
        slice_data = trigger_slices[state['slice_idx']]
        time_slice = trigger_times[state['slice_idx']]
        time_slice_ms = np.array(time_slice, dtype=float) * 1000.0
        for ch_idx in range(num_channels):
            line, = ax.plot(time_slice_ms, slice_data[:, ch_idx], alpha=0.5, linewidth=2)
            state['lines'].append(line)
            state['line_indices'].append(ch_idx)
        
        ax.set_xlabel('Time (ms)')
        ax.set_ylabel('Current (A)')
        set_plot_titles(
            f"Trigger {state['slice_idx']} - All Channels",
            "Hover to identify, click to view that channel's triggers",
            " "
        )
        ax.grid(True)
        x_start, x_end = get_adc_time_limits(time_slice)
        x_start *= 1000.0
        x_end *= 1000.0
        x_range = x_end - x_start
        ax.set_xlim(x_start - 0.05 * x_range, x_end + 0.05 * x_range)
    
    # Always set to global y-axis limits
    ax.set_ylim(global_ylim)
    update_widget_visibility()
    update_toggle_button_style()
    
    # Create new annotation
    state['annotation'] = ax.annotate('', xy=(0,0), xytext=(20,20), textcoords='offset points',
                                      bbox=dict(boxstyle='round', fc='yellow', alpha=0.9),
                                      arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'),
                                      visible=False, fontsize=12, weight='bold')
    state['highlighted_line'][0] = None
    fig.canvas.draw_idle()

def on_hover_toggle(event):
    """Handle hover events"""
    if event.inaxes != ax:
        if state['annotation']:
            state['annotation'].set_visible(False)
        if state['highlighted_line'][0] is not None:
            state['highlighted_line'][0].set_linewidth(2)
            state['highlighted_line'][0].set_alpha(0.5)
            state['highlighted_line'][0] = None
        fig.canvas.draw_idle()
        return
    
    # Check if hovering over any line
    found = False
    for i, line in enumerate(state['lines']):
        if line.contains(event)[0]:
            found = True
            # Reset previous highlight
            if state['highlighted_line'][0] is not None and state['highlighted_line'][0] != line:
                state['highlighted_line'][0].set_linewidth(2)
                state['highlighted_line'][0].set_alpha(0.5)
            
            # Highlight current line
            line.set_linewidth(3)
            line.set_alpha(1.0)
            state['highlighted_line'][0] = line
            
            # Update annotation
            if state['mode'] == 'triggers':
                trigger_idx = state['line_indices'][i]
                label = f"Trigger {trigger_idx}"
            else:
                label = f"Channel {state['line_indices'][i]}"
            state['annotation'].set_text(label)
            state['annotation'].xy = (event.xdata, event.ydata)
            state['annotation'].set_visible(True)
            fig.canvas.draw_idle()
            break
    
    if not found:
        state['annotation'].set_visible(False)
        if state['highlighted_line'][0] is not None:
            state['highlighted_line'][0].set_linewidth(2)
            state['highlighted_line'][0].set_alpha(0.5)
            state['highlighted_line'][0] = None
        fig.canvas.draw_idle()

def on_click_toggle(event):
    """Handle click events to toggle modes"""
    if event.inaxes != ax:
        return
    
    # Check if clicking on any line
    for i, line in enumerate(state['lines']):
        if line.contains(event)[0]:
            # Toggle mode and update indices
            if state['mode'] == 'triggers':
                # Switch to channels mode for clicked trigger
                state['mode'] = 'channels'
                state['slice_idx'] = state['line_indices'][i]
                sync_chan_slider_to_slice()
            else:
                # Switch to triggers mode for clicked channel
                state['mode'] = 'triggers'
                state['channel_idx'] = state['line_indices'][i]
                state['trigger_idx'] = state['slice_idx']
                sync_channel_slider_to_idx()
                sync_trig_slider_to_trigger()
            
            update_plot()
            break

def on_toggle_button(_event):
    """Handle button toggle for all triggers"""
    state['show_all_triggers'] = not state['show_all_triggers']
    update_plot()

def on_trig_change(_value):
    """Handle slider changes for trigger selection"""
    state['trigger_idx'] = int(state['trig_slider'].val)
    sync_trig_textbox_to_trigger()
    update_plot()

def on_trig_minus_clicked(_event):
    new_val = max(0, int(state['trig_slider'].val) - 1)
    state['trig_slider'].set_val(new_val)

def on_trig_plus_clicked(_event):
    new_val = min(len(trigger_slices) - 1, int(state['trig_slider'].val) + 1)
    state['trig_slider'].set_val(new_val)

def on_trig_submit(text):
    try:
        new_val = int(text)
    except ValueError:
        sync_trig_textbox_to_trigger()
        return
    new_val = max(0, min(len(trigger_slices) - 1, new_val))
    state['trig_slider'].set_val(new_val)

def on_channel_change(_value):
    state['channel_idx'] = int(state['channel_slider'].val)
    sync_channel_textbox_to_idx()
    if state['mode'] == 'triggers':
        update_plot()

def on_channel_minus_clicked(_event):
    new_val = max(0, int(state['channel_slider'].val) - 1)
    state['channel_slider'].set_val(new_val)

def on_channel_plus_clicked(_event):
    new_val = min(num_channels - 1, int(state['channel_slider'].val) + 1)
    state['channel_slider'].set_val(new_val)

def on_channel_submit(text):
    try:
        new_val = int(text)
    except ValueError:
        sync_channel_textbox_to_idx()
        return
    new_val = max(0, min(num_channels - 1, new_val))
    state['channel_slider'].set_val(new_val)

def on_chan_change(_value):
    state['slice_idx'] = int(state['chan_slider'].val)
    sync_chan_textbox_to_slice()
    update_plot()

def on_chan_minus_clicked(_event):
    new_val = max(0, int(state['chan_slider'].val) - 1)
    state['chan_slider'].set_val(new_val)

def on_chan_plus_clicked(_event):
    new_val = min(len(trigger_slices) - 1, int(state['chan_slider'].val) + 1)
    state['chan_slider'].set_val(new_val)

def on_chan_submit(text):
    try:
        new_val = int(text)
    except ValueError:
        sync_chan_textbox_to_slice()
        return
    new_val = max(0, min(len(trigger_slices) - 1, new_val))
    state['chan_slider'].set_val(new_val)

# Initial plot
update_plot()

# Connect events
fig.canvas.mpl_connect('motion_notify_event', on_hover_toggle)
fig.canvas.mpl_connect('button_press_event', on_click_toggle)
state['toggle_button'].on_clicked(on_toggle_button)
state['channel_slider'].on_changed(on_channel_change)
state['channel_minus_button'].on_clicked(on_channel_minus_clicked)
state['channel_plus_button'].on_clicked(on_channel_plus_clicked)
state['channel_box'].on_submit(on_channel_submit)
state['trig_slider'].on_changed(on_trig_change)
state['trig_minus_button'].on_clicked(on_trig_minus_clicked)
state['trig_plus_button'].on_clicked(on_trig_plus_clicked)
state['trig_box'].on_submit(on_trig_submit)
state['chan_slider'].on_changed(on_chan_change)
state['chan_minus_button'].on_clicked(on_chan_minus_clicked)
state['chan_plus_button'].on_clicked(on_chan_plus_clicked)
state['chan_box'].on_submit(on_chan_submit)

plt.show()
